In [2]:
import pandas as pd
import numpy as np
import mysql.connector

In [14]:
#Existing columns
columns = ['Investor', 'Series', '%', 'Opening Allocation Value', 'Subscriptions',
       'Redemptions', 'Crystalised Performance Fee', 'Switch',
       'UNEVEN ALLOCATIONS AFFECTING CAPITAL',
       'Closing Value after Subs & Reds', 'Other Income & Expenses',
       'UNEVEN ALLOCATIONS', 'Closing NAV berfore % Based Fees',
       'Administration Fee', 'Closing NAV before MGMT Fee', 'Management Fee',
       'Closing NAV before Performance Fee', 'Performance Fee', 'Closing NAV ']

#File name
file = 'Polar_Star_Fund_Ltd-Rebate(Advisor)-Abdullah-July-2019.xlsx'

# Load all three sheets.
df_admin = pd.read_excel(file,sheet_name='Admin File',skiprows=7)
df_advisor = pd.read_excel(file,sheet_name='Advisor Split')
df_key = pd.read_excel(file,sheet_name='KEY')

# Choose specified columns from admin sheet 
df_admin = pd.DataFrame(data = df_admin, columns = columns)
df_advisor = pd.DataFrame(data = df_advisor,columns = ['Fee'] + columns)

# Find the data
df_admin = df_admin[df_admin.Investor.notnull() & df_admin.Series.notnull()].reset_index(drop=True)
df_advisor = df_advisor[df_advisor.Investor.notnull() & df_advisor.Series.notnull()].reset_index(drop=True)

# Left join: Take items from left table (admin) and (only) matching items from rght table (advisor)
# In this case we take all the columns in admin and join to right table (advisor) on investor,series 

df_join = pd.merge(df_admin,df_advisor, on=['Investor','Series'],how='left',suffixes=(' ',' '))
#Remove spaces at beggining and end of column names
df_join.columns = df_join.columns.str.strip()
df_key.index = df_key.index.str.strip()

#Remove duplicate columns resulting from left join
df_join = df_join.loc[:,~df_join.columns.duplicated()]

#Move fees column to beginning
df_join = df_join[['Fee'] + [col for col in df_join.columns if col != 'Fee']]

#Display database
Advisor = 'Ashburton'
Date = '31-Jul-19'
MngFee = (1 - 0.5*df_key.loc[Advisor]['Mgnt Fee'])
PerfFee = (1 - 0.05*df_key.loc[Advisor]['Perf. Fee'])

df1 = df_join[df_join['Fee'] == Advisor].reset_index(drop = True)

df2 = pd.DataFrame(data = [[df1['Management Fee'].sum(),df1['Performance Fee'].sum()]],
                   columns =['Management Fee Total (excl Vat)','Performance Fee Total (excl Vat)'],index = [Date])

df3 = pd.DataFrame(data = [[MngFee*df1['Management Fee'].sum(),
                           PerfFee*df1['Performance Fee'].sum()]],
                            columns = ['Management Fee payable (' + str(MngFee*100) +'%) excl Vat', 
                                       'Performance Fee payable (' + str(PerfFee*100) + '%) excl Vat'],index=df2.index)

writer = pd.ExcelWriter(Advisor + ' ' + Date + ".xlsx", engine='xlsxwriter')

df1.to_excel(writer, sheet_name=Date)
df2.to_excel(writer, sheet_name= Advisor + ' Fees')
df3.to_excel(writer, sheet_name= Advisor + ' Fees',startrow = 7)

workbook  = writer.book
worksheet1 = writer.sheets[Date]
worksheet2 = writer.sheets[Advisor + ' Fees']

#formatting
format1 = workbook.add_format({'num_format': '$#,##0.00'})

worksheet1.set_column('B:U', 18, format1)
worksheet2.set_column('B:C', 18, format1)
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color': '#D7E4BC',
    'border': 1})

#Colours headings
def colour(df,worksheet,row):
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(row, col_num + 1, value, header_format)

colour(df1,worksheet1,0)
colour(df2,worksheet2,0)
colour(df3,worksheet2,4)
writer.save()


In [ ]:
# Take as is, now you know how to do this st0ff
#
# df_join['Management Fee'] = df_key.loc[df_join['Fee']]['Mgnt Fee'].values/100 * df_join['Closing NAV before MGMT Fee']
# df_join['Performance Fee'] = df_key.loc[df_join['Fee']]['Perf. Fee'].values/100 * df_join['Closing NAV before Performance Fee']